In [ ]:
import os
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install -q google-genai google-auth python-dotenv numpy
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
else:
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass

import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kgweber-cwru/coding-with-ai-wn26/blob/main/week-6-best-practices/assignment.ipynb)

# Week 6 Assignment: Polish and Present Your Project

## Objective
Take one of your previous projects (or create a new one) and make it production-ready by adding error handling, testing, cost tracking, and documentation.

## Requirements
1. Choose or build a complete application
2. Add robust error handling
3. Implement cost tracking
4. Create a test suite
5. Write clear documentation
6. Prepare a brief presentation

In [ ]:
import os
import time
from dotenv import load_dotenv
from google import genai
from google.genai import types
import google.auth
from google.api_core import exceptions

load_dotenv('../.env')
creds, project = google.auth.default()
client = genai.Client(vertexai=True, project=project, location="us-central1")

## Step 1: Project Description

# [Your Project Title]

## Overview
[Brief description of what your application does]

## Use Case
[Who is this for and what problem does it solve?]

## Key Features
1. [Feature 1]
2. [Feature 2]
3. [Feature 3]

## Technical Approach
[What techniques/patterns are you using?]
- Prompt engineering / Conversations / RAG / etc.
- Model(s) used
- Key design decisions

## Step 2: Core Application Code

Build or refine your application here. Include comments and clear structure:

In [ ]:
# YOUR APPLICATION CODE HERE

class YourApplication:
    """Main application class"""
    
    def __init__(self):
        # Initialize
        pass
    
    # Add your methods
    pass


## Step 3: Add Error Handling

Wrap your API calls with proper error handling:

In [ ]:
def robust_api_call(contents, max_retries=3, **kwargs):
    """Make API call with retry logic"""
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=kwargs.get('model', 'gemini-2.5-flash-lite'),
                contents=contents,
                config=types.GenerateContentConfig(
                    temperature=kwargs.get('temperature', 0.7),
                    max_output_tokens=kwargs.get('max_output_tokens', None)
                )
            )
            return response
        except exceptions.ResourceExhausted:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"Rate limit. Waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise
        except exceptions.GoogleAPICallError as e:
            if attempt < max_retries - 1:
                time.sleep(1)
            else:
                raise
    raise Exception("Max retries exceeded")

# Integrate this into your application

## Step 4: Add Cost Tracking

In [ ]:
class CostTracker:
    PRICING = {
        "gemini-2.5-flash-lite": {"input": 0.075, "output": 0.30},
        "gemini-1.5-pro": {"input": 3.50, "output": 10.50},
    }
    
    def __init__(self):
        self.total_cost = 0.0
        self.calls = []
    
    def track(self, response, model="gemini-2.5-flash-lite"):
        if not response.usage_metadata:
            return 0.0
            
        input_tokens = response.usage_metadata.prompt_token_count
        output_tokens = response.usage_metadata.candidates_token_count
        
        # Default pricing
        pricing = self.PRICING.get(model, self.PRICING["gemini-2.5-flash-lite"])
        
        cost = (
            (input_tokens / 1_000_000) * pricing["input"] +
            (output_tokens / 1_000_000) * pricing["output"]
        )
        
        self.total_cost += cost
        self.calls.append({"cost": cost, "tokens": input_tokens + output_tokens})
        return cost
    
    def report(self):
        return f"Total cost: ${self.total_cost:.6f} across {len(self.calls)} calls"

# Create tracker and use in your application
cost_tracker = CostTracker()

## Step 5: Create Test Suite

In [ ]:
class TestSuite:
    def __init__(self, app):
        self.app = app
        self.tests = []
        self.results = []
    
    def add_test(self, name, test_func, expected):
        """Add a test case"""
        self.tests.append({
            "name": name,
            "func": test_func,
            "expected": expected
        })
    
    def run(self):
        """Run all tests"""
        print(f"Running {len(self.tests)} tests...\n")
        passed = 0
        
        for test in self.tests:
            try:
                result = test["func"]()
                success = self.check_result(result, test["expected"])
                
                if success:
                    print(f"✓ {test['name']}")
                    passed += 1
                else:
                    print(f"✗ {test['name']}")
                    print(f"  Expected: {test['expected']}")
                    print(f"  Got: {result}")
            except Exception as e:
                print(f"✗ {test['name']} - ERROR: {e}")
        
        print(f"\n{passed}/{len(self.tests)} tests passed")
    
    def check_result(self, result, expected):
        """Check if result meets expectations"""
        if isinstance(expected, list):
            # Check if any expected string is in result
            return any(exp.lower() in str(result).lower() for exp in expected)
        return expected in str(result)

# Create and populate test suite
# suite = TestSuite(your_app)
# suite.add_test("Test 1", lambda: your_app.method(), "expected output")
# suite.run()

## Step 6: Define Your Tests

In [ ]:
# YOUR TESTS HERE

# Example:
# def test_basic_functionality():
#     result = your_app.main_method("test input")
#     return result

# suite.add_test(
#     "Basic functionality",
#     test_basic_functionality,
#     expected=["expected", "keywords"]
# )

## Step 7: Demonstrate Your Application

Show your application working with real examples:

In [ ]:
print("="*70)
print("APPLICATION DEMONSTRATION")
print("="*70)

# Demo 1
print("\nExample 1: [Describe what this shows]")
# YOUR DEMO CODE

# Demo 2
print("\nExample 2: [Describe what this shows]")
# YOUR DEMO CODE

# Demo 3
print("\nExample 3: [Describe what this shows]")
# YOUR DEMO CODE

# Show cost report
print("\n" + "="*70)
print("COST REPORT")
print("="*70)
print(cost_tracker.report())

## Step 8: Documentation

### User Guide

#### How to Use
1. [Step 1]
2. [Step 2]
3. [Step 3]

#### Examples
```python
# Example usage code
```

### Technical Documentation

#### Architecture
[Describe how your system is structured]

#### Key Components
- Component 1: [description]
- Component 2: [description]

#### API/Methods
Document your main methods:
- `method_name(param1, param2)`: [what it does]

### Known Limitations
1. [Limitation 1]
2. [Limitation 2]

### Future Improvements
1. [Improvement 1]
2. [Improvement 2]

## Step 9: Reflection and Analysis

### Project Reflection

#### What went well?


**YOUR ANSWER**

#### What was challenging?

**YOUR ANSWER**

#### What did you learn?

**YOUR ANSWER**

#### How would you improve this for production?

**YOUR ANSWER**

#### What would you build next?

**YOUR ANSWER**

## Step 10: Presentation Prep

Prepare a 5-minute presentation covering:

### Slide 1: Title
- Project name
- Your name
- One sentence description

### Slide 2: Problem & Solution
- What problem does this solve?
- Who is it for?
- Why LLMs/AI?

### Slide 3: Technical Approach
- Architecture overview
- Key techniques used
- Design decisions

### Slide 4: Demo
- Live demonstration or screenshots
- Show key features
- Highlight what makes it useful

### Slide 5: Results & Next Steps
- What worked well?
- Metrics (cost, performance, etc.)
- Future improvements
- Lessons learned

## Submission Checklist

- [ ] Complete working application
- [ ] Error handling implemented
- [ ] Cost tracking working
- [ ] Test suite created and run
- [ ] Clear documentation
- [ ] Demonstrations with real examples
- [ ] Reflection questions answered
- [ ] Presentation prepared
- [ ] Code is well-commented
- [ ] Ready to present!

## Congratulations!

You've completed the entire workshop series and built a production-ready LLM application!

### What You've Accomplished:
- ✓ Mastered LLM API fundamentals
- ✓ Built conversational systems
- ✓ Created reusable prompt templates
- ✓ Implemented semantic search with embeddings
- ✓ Built complete RAG systems
- ✓ Applied production best practices
- ✓ Created a polished, documented application

### Keep Building!
The field is evolving rapidly. Stay curious, keep experimenting, and share what you create!